In [1]:
from src.data.dataloaders import GMC_DataLoader, NIH_DataLoader
from src.models.train import train, validate
from src.models.evaluate import test
from src.models.CheXNet import DenseNet121
from src.models.baseline import Baseline
from src.models.unet import UNet
from src.models.style_encoder import ConvEncoder
from src.models.super_model import SuperModel
from src.models.resnet import get_resnet

from torch import optim
import torch

%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
train_loader_GMC = GMC_DataLoader(stage='train', batch_size = 5, shuffle=True, balanced = True, num_workers=0, pin_memory=True,limit=500)
train_loader_NIH= NIH_DataLoader(stage='train', batch_size = 5, shuffle=True, balanced = True, num_workers=0, pin_memory=True,limit=500)
train_loaders = [train_loader_GMC, train_loader_NIH]

val_loader_GMC = GMC_DataLoader(stage='train', batch_size = 5, shuffle=True, balanced = True, num_workers=0, pin_memory=True,limit=500)
val_loader_NIH= NIH_DataLoader(stage='train', batch_size = 5, shuffle=True, balanced = True, num_workers=0, pin_memory=True,limit=500)
val_loaders = [val_loader_GMC, val_loader_NIH]

test_loader_GMC = GMC_DataLoader(stage='test', batch_size = 5, shuffle=True, balanced = True, num_workers=0, pin_memory=True,limit=465)
test_loader_NIH= NIH_DataLoader(stage='test', batch_size = 5, shuffle=True, balanced = True, num_workers=0, pin_memory=True,limit=465)
test_loaders = [test_loader_GMC, test_loader_NIH]


In [3]:
len(test_loader_GMC)

93

In [4]:
super_model = SuperModel()
optimizer = optim.Adam(super_model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

In [5]:
n_epochs = 20
irm = 0.1
vrex = 0

train(super_model, train_loaders, val_loaders, optimizer, criterion, irm, vrex, n_epochs, device, exp2 = True)


 11%|█         | 214/2000 [36:00<3:32:19,  7.13s/it]  

In [20]:
from src.data.dataloaders import ENV_DataLoader
irm = 0.1
vrex = 0
n_epochs = 20

name_model = f'{super_model.name}_irm_{irm}_ep_{n_epochs}'
test(super_model, test_loader_NIH, name_model, device, exp2 = True)

  0%|          | 0/93 [00:00<?, ?it/s]

Accuracy of the model: 49.46236559139785 %


In [ ]:
val_loader_NIH.show()

In [17]:
import random
lines = open(r'data/labels/NIH/balanced_output_test.txt').readlines()
random.shuffle(lines)
open(r'data/labels/NIH/balanced_output_test.txt', 'w').writelines(lines)

In [160]:
import random
with open(r'data/labels/NIH/output_test.txt','r') as source:
    num_zero = 0
    num_one = 0
    for line in source:
        label = line.split(' ')[-1][0]
        if label == '0':
            num_zero += 1
        elif label == '1':
            num_one += 1
    num_sample = min(num_zero,num_one)

with open(r'data/labels/NIH/output_test.txt','r') as source:
    num_zero = 0
    num_one = 0
    lines = []
    for line in source:
        label = line.split(' ')[-1][0]
        if (label == '0') and (num_zero < num_sample):
            lines.append(line)
            num_zero += 1
        elif (label == '1') and (num_one < num_sample):
            lines.append(line)
            num_one += 1

open(r'data/labels/NIH/balanced_output_test.txt', 'w').writelines(lines)

        

In [106]:
num_one

234